In [2]:
import os
import sys
sys.path.append(os.path.realpath('../..'))
import aoc
my_aoc = aoc.AdventOfCode(2020,19)
from solution import strings_for_rule, match_rule, parse_input, parse_rule

In [3]:
input_text = """0: 4 1 5
1: 2 3 | 3 2
2: 4 4 | 5 5
3: 4 5 | 5 4
4: "a"
5: "b"

ababbb
bababa
abbbab
aaabbb
aaaabbb"""


In [ ]:
from heapq import heappop, heappush
import itertools

input_text = """0: 4 1 5
1: 2 3 | 3 2
2: 4 4 | 5 5
3: 4 5 | 5 4
4: "a"
5: "b"

ababbb
bababa
abbbab
aaabbb
aaaabbb"""

def check_message(message, rule_id, rules):
    heap = []
    heappush(heap, (len(rules[rule_id]), (rules[rule_id])))
    counter = 0
    seen = set()
    while heap:
        counter += 1
        if counter > 10:
            return False
        length, rule = heappop(heap)
        print(counter, length, rule)

        # discard rule
        if length > len(message):
            print(f"discard length: {rule}")
            continue

        if rule in seen:
            print(f"discard seen: {rule}")
            continue
        seen.add(rule)

        if isinstance(rule, str):
            if rule == message:
                return True

        # Case 1:  all items are strings
        if all(isinstance(item, str) for item in rule):
            print(f"case 1: {rule}")
            # check to see if one of the strings is our message
            for item in rule:
                if item == message:
                    return True
            # no further processing on this chain
            continue

        # case 2: all items are ints:
        if all(isinstance(item, int) for item in rule):
            print(f"case 2: {rule}")
            # convert ints to their rules
            heappush(heap, (length, tuple(rules[item] for item in rule)))
            continue
        
        # case 3: tuple(tuple(str))
        if not has_ints(rule):
            print(f"case 3: {rule}")
            # ('a', (((('a', 'a'), ('b', 'b')), (('a', 'b'), ('b', 'a'))), ((('a', 'b'), ('b', 'a')), (('a', 'a'), ('b', 'b')))), 'b')
            new_rule = find_strings(rule)
            if new_rule != rule:
                heappush(heap, (len(new_rule), new_rule))
            for option in build_strings(rule):
                heappush(heap, (len(option), option))


        # case 4: mixed unresolved rule
        print(f"case 4: {rule}")
        new_rule = resolve_rule(rule, rules)
        heappush(heap, (len(new_rule), new_rule))           
    return False

def resolve_rule(rule, rules, limit=20):
    if isinstance(rule, str):
        return rule
    
    if isinstance(rule,int):
        return rules[rule]
    
    if len(rule) > limit:
        return None
    new_rule = []
    for item in rule:
        if isinstance(item, str):
            new_rule.append(item)
        elif isinstance(item, int):
            # replace rule
            new_rule.append(rules[item])
        else:
            new_rule.append(resolve_rule(item, rules, limit))
    if len(new_rule) == 1:
        new_rule = resolve_rule(new_rule[0], rules, limit)
    return tuple(new_rule)
        
def has_ints(rule):
    if isinstance(rule, str):
        return False
    if isinstance(rule, int):
        return True
    int_detected = False
    for item in rule:
        if isinstance(item, str):
            continue
        if isinstance(item,int):
            int_detected = True
            continue
        if has_ints(item):
            int_detected = True
    return int_detected
        

def expand_rule(rule):
    new_rule = list(rule)
    new_rules = []
    for idx, item in enumerate(rule):
        if isinstance(item, tuple):
            while len(item) == 1:
                item = item[0]
            for element in item:
                new_rule[idx] = element
                new_rules.append(tuple(new_rule)) 
            break
    if not new_rules:
        new_rules = [rule]
    return new_rules

def find_strings(input_tuple):
    working_list = list(input_tuple)
    if all(isinstance(item, str) for item in working_list):
        return (''.join(working_list),)
    for idx, item in enumerate(working_list):
        if isinstance(item, tuple):
            working_list[idx] = find_strings(item)
    return tuple(working_list)




# for idx in (5,4,3,2,1,0):
#     print(idx, resolve_rule(rules[idx], rules))

rules, messages = parse_input(input_text)

# new_rule = resolve_rule(rules[0], rules)
# print(new_rule)
# while has_ints(new_rule):
#     new_rule = resolve_rule(new_rule, rules)
#     print(new_rule)

# print()
# new_rules = expand_rule(new_rule)
# for new_rule in new_rules:
#     print(new_rule)
    
for rule_id in (1,):
    print(rule_id)
    print(f"{rule_id}: {check_message('ababbb', rule_id, rules)}")

# for rule in expand_rule(resolve_rule(rules[3], rules)):
#     print(expand_rule(rule))

# expand_rule(('a', 'b'))

check_message('aabb', 0, rules)

1
1 2 ((2, 3), (3, 2))
case 4: ((2, 3), (3, 2))
2 2 ((((4, 4), (5, 5)), ((4, 5), (5, 4))), (((4, 5), (5, 4)), ((4, 4), (5, 5))))
case 4: ((((4, 4), (5, 5)), ((4, 5), (5, 4))), (((4, 5), (5, 4)), ((4, 4), (5, 5))))
3 2 (((('a', 'a'), ('b', 'b')), (('a', 'b'), ('b', 'a'))), ((('a', 'b'), ('b', 'a')), (('a', 'a'), ('b', 'b'))))
case 3: (((('a', 'a'), ('b', 'b')), (('a', 'b'), ('b', 'a'))), ((('a', 'b'), ('b', 'a')), (('a', 'a'), ('b', 'b'))))
build_strings((((('a', 'a'), ('b', 'b')), (('a', 'b'), ('b', 'a'))), ((('a', 'b'), ('b', 'a')), (('a', 'a'), ('b', 'b')))))
  aaaa
  aaab
  aaba
  aabb
  abaa
  abab
  abba
  abbb
  baaa
  baab
  baba
  babb
  bbaa
  bbab
  bbba
  bbbb
case 4: (((('a', 'a'), ('b', 'b')), (('a', 'b'), ('b', 'a'))), ((('a', 'b'), ('b', 'a')), (('a', 'a'), ('b', 'b'))))
4 2 (((('a', 'a'), ('b', 'b')), (('a', 'b'), ('b', 'a'))), ((('a', 'b'), ('b', 'a')), (('a', 'a'), ('b', 'b'))))
discard seen: (((('a', 'a'), ('b', 'b')), (('a', 'b'), ('b', 'a'))), ((('a', 'b'), ('b', '

TypeError: can only concatenate str (not "tuple") to str

In [47]:
def build_strings(input_tuple):
    print(f"build_strings({input_tuple})")
    partial_results = [""]
    # (((('aa',), ('bb',)), (('ab',), ('ba',))), ((('ab',), ('ba',)), (('aa',), ('bb',))))
    for option in input_tuple:
        # ((('aa',), ('bb',)), (('ab',), ('ba',)))
        print(f"option: {option}")
        for sub_rule in option:
            print(f"sub_rule: {sub_rule}")
            if isinstance(sub_rule, str):
                partial_results = [p_r + sub_rule for p_r in partial_results]
                print(f"partial_results: {partial_results}")
                continue
            if isinstance(sub_rule, tuple):
                # (('aa',), ('bb',))
                for item in sub_rule:
                    sub_strings = build_strings(item)
                    print(f"sub_strings: {sub_strings}")
                    # Generate all combinations of `partial_results` and `sub_strings`.
                    
                    partial_results = [p_r + s_s for p_r in partial_results for s_s in sub_strings]
    for result in partial_results:
        print(f"  {result}")
    return partial_results

test=(
    'a',
    (
        (
            (('a', 'a'), ('b', 'b')),
            (('a', 'b'), ('b', 'a'))
        ),
        (
            (('a', 'b'), ('b', 'a')),
            (('a', 'a'), ('b', 'b'))
        )
    ),
    'b'
)
# a,(aaab),(abba),b


test_2 = find_strings(test)
print(test_2)
test_3 = build_strings(test_2)
print(test_3)


('a', (((('aa',), ('bb',)), (('ab',), ('ba',))), ((('ab',), ('ba',)), (('aa',), ('bb',)))), 'b')
build_strings(('a', (((('aa',), ('bb',)), (('ab',), ('ba',))), ((('ab',), ('ba',)), (('aa',), ('bb',)))), 'b'))
option: a
sub_rule: a
partial_results: ['a']
option: (((('aa',), ('bb',)), (('ab',), ('ba',))), ((('ab',), ('ba',)), (('aa',), ('bb',))))
sub_rule: ((('aa',), ('bb',)), (('ab',), ('ba',)))
build_strings((('aa',), ('bb',)))
option: ('aa',)
sub_rule: aa
partial_results: ['aa']
option: ('bb',)
sub_rule: bb
partial_results: ['aabb']
  aabb
sub_strings: ['aabb']
build_strings((('ab',), ('ba',)))
option: ('ab',)
sub_rule: ab
partial_results: ['ab']
option: ('ba',)
sub_rule: ba
partial_results: ['abba']
  abba
sub_strings: ['abba']
sub_rule: ((('ab',), ('ba',)), (('aa',), ('bb',)))
build_strings((('ab',), ('ba',)))
option: ('ab',)
sub_rule: ab
partial_results: ['ab']
option: ('ba',)
sub_rule: ba
partial_results: ['abba']
  abba
sub_strings: ['abba']
build_strings((('aa',), ('bb',)))
opti

In [ ]:
('a',
 (
     ((('a', 'a'), ('b', 'b')), (('a', 'b'), ('b', 'a'))),
     ((('a', 'b'), ('b', 'a')), (('a', 'a'), ('b', 'b')))),
 'b')

input_text = """0: 4 1 5
1: 2 3 | 3 2
2: 4 4 | 5 5
3: 4 5 | 5 4
4: "a"
5: "b"

ababbb
bababa
abbbab
aaabbb
aaaabbb"""



for key, rule in rules.items():
    strings = strings_for_rule(rule, rules)
    print(key, strings)

for message in messages:
    print(f"message: {message}, matches: {match_rule(message, 0, rules)}")

In [ ]:
input_text="""42: 9 14 | 10 1
9: 14 27 | 1 26
10: 23 14 | 28 1
1: "a"
11: 42 31
5: 1 14 | 15 1
19: 14 1 | 14 14
12: 24 14 | 19 1
16: 15 1 | 14 14
31: 14 17 | 1 13
6: 14 14 | 1 14
2: 1 24 | 14 4
0: 8 11
13: 14 3 | 1 12
15: 1 | 14
17: 14 2 | 1 7
23: 25 1 | 22 14
28: 16 1
4: 1 1
20: 14 14 | 1 15
3: 5 14 | 16 1
27: 1 6 | 14 18
14: "b"
21: 14 1 | 1 14
25: 1 1 | 1 14
22: 14 14
8: 42
26: 14 22 | 1 20
18: 15 15
7: 14 5 | 1 21
24: 14 1

abbbbbabbbaaaababbaabbbbabababbbabbbbbbabaaaa
bbabbbbaabaabba
babbbbaabbbbbabbbbbbaabaaabaaa
aaabbbbbbaaaabaababaabababbabaaabbababababaaa
bbbbbbbaaaabbbbaaabbabaaa
bbbababbbbaaaaaaaabbababaaababaabab
ababaaaaaabaaab
ababaaaaabbbaba
baabbaaaabbaaaababbaababb
abbbbabbbbaaaababbbbbbaaaababb
aaaaabbaabaaaaababaa
aaaabbaaaabbaaa
aaaabbaabbaaaaaaabbbabbbaaabbaabaaa
babaaabbbaaabaababbaabababaaab
aabbbbbaabbbaaaaaabbbbbababaaaaabbaaabba"""

In [ ]:
rules, messages = parse_input(input_text)
rule_cache = {}
counter = 0
for message in messages:
    print(f"message: {message}, matches: {match_rule(message, 0, rules)}")
    if match_rule(message, 0, rules):
        counter += 1
print(counter)

In [ ]:



current = (
    (
        (('a', 'a'), ('b', 'b')),
        (('a', 'b'), ('b', 'a'))
    ), 
    (
        (('a', 'b'), ('b', 'a')),
        (('a', 'a'), ('b', 'b'))
    )
)

new_rec = find_strings(current)
import json 

print(new_rec)

# [[[["aa"], ["bb"]], [["ab"], ["ba"]]], [[["ab"], ["ba"]], [["aa"], ["bb"]]]]

(((('aa',), ('bb',)), (('ab',), ('ba',))), ((('ab',), ('ba',)), (('aa',), ('bb',))))


In [66]:
input_text = """0: 4 1 5
1: 2 3 | 3 2
2: 4 4 | 5 5
3: 4 5 | 5 4
4: "a"
5: "b"

ababbb
bababa
abbbab
aaabbb
aaaabbb"""

from functools import lru_cache

def has_ints(rule):
    if isinstance(rule, str):
        return False
    if isinstance(rule, int):
        return True
    int_detected = False
    for item in rule:
        if isinstance(item, str):
            continue
        if isinstance(item,int):
            int_detected = True
            continue
        if has_ints(item):
            int_detected = True
    return int_detected

def is_resolved(rule):
    return not has_ints(rule)

def can_resolve(rule):
    print(f"can_resolve({type(rule)}{rule})")
    if isinstance(rule, tuple):
        print("tuple")
        return all(can_resolve(sub_rule) for sub_rule in rule)
    if isinstance(rule, str):
        print("string")
        return True
    if isinstance(rule, int) and isinstance(rules[rule], str):
        print(f"int {rule}: {rules[rule]}")
        return True
    print(f"can't resolve {rule}")
    return False

def resolve_rule(rule):
    if isinstance(rule, str):
        return rule
    if isinstance(rule, int):
        return resolve_rule(rules[rule])
    results = []
    for option in rule:
        partial_results = [""]
        for sub_rule in option:
            sub_strings = resolve_rule(sub_rule)
            # Generate all combinations of `partial_results` and `sub_strings`.
            partial_results = [p_r + s_s for p_r in partial_results for s_s in sub_strings]
        if partial_results != [""]:
            results.extend(partial_results)
    return tuple(results)

rules, messages = parse_input(input_text)
for idx, rule in rules.items():
    print(idx, rule, is_resolved(rule), can_resolve(rule))
    if can_resolve(rule):
        rules[idx] = resolve_rule(rule)
print("\nAfter:")
for idx, rule in rules.items():
    print(idx, rule, is_resolved(rule), can_resolve(rule))
    if can_resolve(rule):
        rules[idx] = resolve_rule(rule)
print("\nAfter:")
for idx, rule in rules.items():
    print(idx, rule, is_resolved(rule), can_resolve(rule))



can_resolve(<class 'tuple'>((4, 1, 5),))
tuple
can_resolve(<class 'tuple'>(4, 1, 5))
tuple
can_resolve(<class 'int'>4)
int 4: a
can_resolve(<class 'int'>1)
can't resolve 1
0 ((4, 1, 5),) False False
can_resolve(<class 'tuple'>((4, 1, 5),))
tuple
can_resolve(<class 'tuple'>(4, 1, 5))
tuple
can_resolve(<class 'int'>4)
int 4: a
can_resolve(<class 'int'>1)
can't resolve 1
can_resolve(<class 'tuple'>((2, 3), (3, 2)))
tuple
can_resolve(<class 'tuple'>(2, 3))
tuple
can_resolve(<class 'int'>2)
can't resolve 2
1 ((2, 3), (3, 2)) False False
can_resolve(<class 'tuple'>((2, 3), (3, 2)))
tuple
can_resolve(<class 'tuple'>(2, 3))
tuple
can_resolve(<class 'int'>2)
can't resolve 2
can_resolve(<class 'tuple'>((4, 4), (5, 5)))
tuple
can_resolve(<class 'tuple'>(4, 4))
tuple
can_resolve(<class 'int'>4)
int 4: a
can_resolve(<class 'int'>4)
int 4: a
can_resolve(<class 'tuple'>(5, 5))
tuple
can_resolve(<class 'int'>5)
int 5: b
can_resolve(<class 'int'>5)
int 5: b
2 ((4, 4), (5, 5)) False True
can_resolve(<cl

In [ ]:
# stolen code to review, credit u/Gprinziv:
def makeRules(fn):
  with open(fn) as file:
    raw = file.read().split("\n\n")
  rulesDict = {}
  for rule in raw[0].split("\n"):
    rule = rule.split(":")
    if "\"" in rule[1]:
      subrules = rule[1][2]
    elif "|" in rule[1]:
      subrules = [[int(j)for j in i.strip().split()] for i in rule[1].strip().split("|")]
    else:
      subrules = [int(i) for i in rule[1].strip().split(" ")]
    rulesDict[int(rule[0])] = subrules
  messages = raw[1].split("\n")
  # print(rulesDict)
  return rulesDict, messages

def printRules(rulesDict):
  print("Printing Rules")
  for rule in rulesDict:
    print(str(rule) + ": " + str(rulesDict[rule]))
  print("Done.")

#Takes a formatted password string and calculates the length.
#Assumes all branches of the password are uniform in length.
#Input: "a[[aa|bb][ab|ba]|[ab|ba][aa|bb]]b"
#Output: 6
def getLen(rule): #Computes the length of password rules for isvalid.
  cur, count = 0, 0
  while cur < len(rule):
    if rule[cur] in ("a", "b"):
      count += 1
    elif rule[cur] == "|":
      depth = 1
      while depth > 0:
        cur += 1
        if rule[cur] == "|":
          depth += 1
        elif rule[cur] == "]":
          depth -= 1
    cur += 1

  return count

#Determines if a given message is valid against the formatted pasword string
#Inputs: (String) formatted password string, (String) message
#Output: Boolean 
def isValid(rule, message):
  ptrs = [0]
  mPtr = [0]
  ruleLen = getLen(rule)

  if len(message) != ruleLen:
    return False

  #Because we know that the length of the message matches the check exactly we know that once you get past the nth letter, you've passed.
  while ptrs:
    if mPtr[0] >= len(message):
      return True

    cur = rule[ptrs[0]] #Get the value pointed to by the current pointer
    if cur.isalpha(): #If it's a simple a or b, evaluate it. If it's bad, trash the pointers and start from the next one.
      if message[mPtr[0]] == cur:
        ptrs[0] += 1
        mPtr[0] += 1
      else:
        ptrs.pop(0)
        mPtr.pop(0)
    elif cur == "[": #Create a second pointer after the relevant "|". Watch for errant "["s. Increment pointer.
      tempPtr, depth = ptrs[0] + 1, 1
      while depth > 0:
        if rule[tempPtr] == "[":
          depth += 1
        elif rule[tempPtr] == "|":
          depth -= 1
        tempPtr += 1
      ptrs.append(tempPtr)
      mPtr.append(mPtr[0])
      ptrs[0] += 1
    elif cur == "|": # Skip forward to after the relevant "]". Watch for errant "["s.
      depth = 1
      while depth > 0:
        ptrs[0] += 1
        if rule[ptrs[0]] == "[":
          depth += 1
        elif rule[ptrs[0]] == "]":
          depth -= 1
      ptrs[0] += 1
    else: #Cur is at a "]", just increment it.
      ptrs[0] += 1

  #If you're run out of valid pointers to check, the rule must be invalid.
  return False

#Takes a set of rules and creates a formatter password string.
#Input: [4, 1, 5]
#Output: "a[[aa|bb][ab|ba]|[ab|ba][aa|bb]]b"
def getRule(ruleIdx):
  global rulesDict
  rule = rulesDict[ruleIdx]
  while not all(type(x) == str for x in rule):
    i = 0
    if type(rule[0]) == list:
        rule = ["["] + rule[0] + ["|"] + rule[1] + ["]"]

    while i < len(rule):
      instruction = rule[i]
      #print(str(i) + ": " + str(instruction))
      if type(instruction) == str:
        pass
      else:
        newInst = rulesDict[instruction]
        if type(newInst) == int or type(newInst) == str:
          rule = rule[:i] + [newInst] + rule[i + 1:]
        else:
          if type(newInst[0]) == int:
            rule = rule[:i] + newInst + rule[i + 1:]
          else:
            rule = rule[:i] + ["["] + newInst[0] + ["|"] + newInst[1] + ["]"] + rule[i + 1:]
      i += 1

  rule = "".join(rule)
  return rule

#Check message segments against rule 42 twice, then check it against 42 until it fails, then check it against 31 until it fails or you pass.
def part2():
  rule42, rule31 = getRule(42), getRule(31)
  ruleLen = getLen(rule42)
  count = 0

  for message in messages:
    if len(message) % ruleLen != 0 or len(message) < (ruleLen * 3):
      pass #Don't even bother validating.
    elif len(message) == (ruleLen * 3):
      count += 1 if isValid(rule42 + rule42 + rule31, message) else 0
    else:
      if isValid(rule42, message[0:ruleLen]) and isValid(rule42, message[ruleLen:2*ruleLen]) and isValid(rule31, message[-ruleLen:]):
        midSegs = int(len(message) / ruleLen) - 3
        valid42 = all(isValid(rule42, message[(2+i)*ruleLen:(3+i)*ruleLen]) for i in range(int(midSegs/2) + midSegs%2))

        flag = 42
        for i in range(int(midSegs/2), 0, -1):
          if flag == 42: 
            flag = 42 if isValid(rule42, message[(i+1)*-ruleLen:i*-ruleLen]) else 31
          if flag == 31:
            flag = 31 if isValid(rule31, message[(i+1)*-ruleLen:i*-ruleLen]) else 0
        count += 1 if flag > 0 and valid42 else 0
      else:
        pass
  return count


rulesDict, messages = makeRules("input.txt")
rule0 = getRule(0)
# print(rule0)
# print(getLen(rule0))
total = sum(isValid(rule0, message) for message in messages)
print("Final count: " + str(total))

p2count = part2()
print("Final count: " + str(p2count))

[a[b[[[b[bba|a[ba|bb]]|a[a[aa|bb]|b[[b|a]a|ab]]]b|[a[aba|[b[b|a]|ab]b]|b[b[b[b|a]|ab]|a[[b|a]a|ab]]]a]a|[b[a[[ba|ab]a|[aa|bb]b]|b[[ab|bb]b|[b[b|a]|ab]a]]|a[a[b[a[b|a]|bb]|aab]|b[baa|a[[b|a]a|ab]]]]b]|a[a[[bbaa|a[[ba|bb]b|[ba|aa]a]]a|[baab|a[baa|a[[b|a]a|ab]]]b]|b[[a[a[a[b|a]|bb]|b[ba|aa]]|b[[ba|aa]a|abb]]a|[[a[ab|aa]|b[ba|aa]]b|[a[aa|bb]|b[[b|a]a|ab]]a]b]]]|b[[[b[[aba|b[aa|bb]]a|[[[b|a]a|ab]b|[a[b|a]|bb]a]b]|a[a[baa|[ba|bb]b]|b[b[b|a][b|a]|a[b[b|a]|ab]]]]a|[[b[b[b|a][b|a]|a[b[b|a]|ab]]|a[b[[b|a]a|ab]|aaa]]b|[a[[ba|aa]a|abb]|b[[ab|aa]b|baa]]a]b]a|[[b[[aba|abb]a|[b[ba|bb]|aaa]b]|a[b[b[b|a][b|a]|a[b[b|a]|ab]]|a[aab|b[ab|bb]]]]a|[a[b[[ab|aa]a|[ba|ab]b]|a[b[[b|a]a|ab]|aab]]|b[b[b[ba|bb]|aaa]|a[b|a][ba|ab]]]b]b]][a[b[[[b[bba|a[ba|bb]]|a[a[aa|bb]|b[[b|a]a|ab]]]b|[a[aba|[b[b|a]|ab]b]|b[b[b[b|a]|ab]|a[[b|a]a|ab]]]a]a|[b[a[[ba|ab]a|[aa|bb]b]|b[[ab|bb]b|[b[b|a]|ab]a]]|a[a[b[a[b|a]|bb]|aab]|b[baa|a[[b|a]a|ab]]]]b]|a[a[[bbaa|a[[ba|bb]b|[ba|aa]a]]a|[baab|a[baa|a[[b|a]a|ab]]]b]|b[[a[a[a[b|a]|bb]|b[ba